## Clone GICAF

In [ ]:
!git clone https://github.com/gasim97/gicaf.git

## Install dependencies
IMPORTANT: before proceeding, restart the colab runtime after running this cell for the first time in a session

NOTE: Brevitas, a dependency, was in alpha stage at the time of writing, as such usage of Brevitas is not considered stable

In [ ]:
import gicaf.Dependencies
gicaf.Dependencies.install()

## Imports

In [ ]:
from gicaf.LoadData import LoadData
import gicaf.models.TfLiteModels as tlmodels
from gicaf.attacks.AdaptiveSimBA import AdaptiveSimBA
from gicaf.AttackEngine import AttackEngine
from gicaf.Logger import Logger
import gicaf.Utils as utils
import matplotlib.pyplot as plt
from os.path import abspath
import logging
logging.basicConfig(level=logging.INFO)

## Load experiment data from Google Drive


In [ ]:
utils.load_tmp_from_gdrive()

## Set-up model

### TfLite

In [ ]:
model = tlmodels.EfficientNetB7(bit_width=8)

## Load data

In [ ]:
parentdir = abspath('')
loadData = LoadData(ground_truth_file_path=parentdir + "/gicaf/data/val.txt", img_folder_path=parentdir + "/gicaf/data/ILSVRC2012_img_val/")

### Load and preprocess data

In [ ]:
data_generator = loadData.get_data([(100, 120)], model.metadata)

### Save preprocessed data

In [ ]:
loadData.save(data_generator, "ILSVRC2012_val_100_to_120_EfficientNetB7")

### Load saved preprocessed data

In [ ]:
data_generator = loadData.load("ILSVRC2012_val_100_to_120_EfficientNetB7", [(7, 7)])

## Run attack

In [ ]:
attacks = [
  AdaptiveSimBA(size=8, epsilon=64/255)
]

In [ ]:
metrics = [
  'absolute-value norm', 
  'psnr', 
  'ssim'
]

In [ ]:
attack_engine = AttackEngine(data_generator, model, attacks)
loggers, success_rates = attack_engine.run(metric_names=metrics)
attack_engine.close() # save experiment logs

In [ ]:
success_rates

## Analyse data

In [ ]:
logger = Logger()
logger.load(1)

In [ ]:
logs = logger.get_all()

In [ ]:
plt.figure(figsize=(10,7))
with plt.style.context('seaborn-whitegrid'):
  plt.plot(logs[0]['ssim'])

## Save experiment data to Google Drive

In [ ]:
utils.save_tmp_to_gdrive()